In [ ]:
# default_exp modelos

# modelos

> Herramientas de ajuste, validación y evaluación de modelos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from functools import reduce
from itertools import chain
import random
import string
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from criminologia_cdmx.etl import *
from criminologia_cdmx.covariables import *

## variable_independiente

In [ ]:
# export
def variable_independiente(datos, columna_y, valores_y, 
                           fecha_inicio, fecha_fin, 
                           agregacion='colonias',
                           nombre_y=None):
    """ Regresa un DataFrame con la variable independicente agregada entre fecha_inicio y fecha_fin
        en las unidades requeridas.
        
        Args:
            datos (DataFrame): carpetas/victimas con ids espaciales y categorías de usuario
            columna_y (str): Nombre de la columna en donde vienen los incidentes de valor_y
            valores_y (list): delitos o categorías a utilizar como Y
            fecha_inicio (str): fecha inicial para agregar delitos "d-m-Y"
            fecha_fin (str): fecha final para agregar delitos "d-m-Y"
            agregacion (str): colonias/cuadrantes. Eventualmente debe recibir 
                              agregaciones arbitrarias (opcional)
            nombre_y (str): Nombre para la columna con la variable dependiente 
                           (opcional, si se omite se concatenan los nombres de valores_y).
    """
    fecha_inicio = pd.to_datetime(fecha_inicio, dayfirst=True)
    fecha_fin = pd.to_datetime(fecha_fin, dayfirst=True)
    datos = datos.loc[datos['fecha_hechos'].between(fecha_inicio, fecha_fin)]
    datos = datos.loc[datos[columna_y].isin(valores_y)]
    if agregacion == 'colonias':
        columna_agrega = 'colonia_cve'
        layer = 'colonias'
    elif agregacion == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
        layer = 'cuadrantes'
    else:
        raise ValueError("unidades debe ser 'colonias' o 'cuadrantes'")
    datos = datos.groupby(columna_agrega).size()
    if nombre_y is not None:
        datos.name = nombre_y
    else:
        datos.name = " ".join(valores_y)
    
    unidades = gpd.read_file("datos/criminologia_capas.gpkg", layer=layer)
    datos = unidades[[columna_agrega]].merge(datos, on=columna_agrega, how='left').fillna(0)
    return datos   

In [ ]:
carpetas = get_carpetas_from_api(100000)
carpetas = agrega_ids_espaciales(carpetas)
fecha_inicio = carpetas.fecha_hechos.min().strftime("%d-%m.%Y")
fecha_fin = carpetas.fecha_hechos.max().strftime("%d-%m.%Y")
# Prueba con un delito
delito = ['ROBO A CASA HABITACION SIN VIOLENCIA']
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin, 
                           agregacion='cuadrantes')
assert delito[0] in Y.columns
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin)
assert delito[0] in Y.columns
# Prueba con dos delitos y nombre de columna
delitos = ['ROBO A CASA HABITACION SIN VIOLENCIA', 'ROBO A CASA HABITACION CON VIOLENCIA']
nombre_y = "Robo a Casa habitación (CV/SV)"
Y = variable_independiente(carpetas, 'delito', delitos, fecha_inicio, fecha_fin, 
                           agregacion='cuadrantes', nombre_y=nombre_y)
assert nombre_y in Y.columns
Y = variable_independiente(carpetas, 'delito', delitos, fecha_inicio, fecha_fin, 
                           nombre_y=nombre_y)
assert nombre_y in Y.columns

## CapaDeAnalisis

In [ ]:
# export
class CapaDeAnalisis(object):
    """ Clase para contener variable objetivo y covariables.
    
        Args:
            Y (DataFrame): debe tener dos columnas: el identificador de la unidad de análisis y 
                           el valor de la variable dependiente 
                           (las columnas deben venir en ese orden).
            covariables (DataFrame): debe contener una columna con el identificador de la unidad 
                                     de análisis (común a Y) y tantas como covariables.
            campo_id (str): el nombre del campo común en X y Y para unirlos.
        Atributos:
            Y (DataFrame): la variable dependiente.
            Y_nombre (str): Nombre de la columna con el delito a modelar.
            X (DataFrame): las variables independientes.
            X_nombres (list): Lista de los nombres de columnas de las covariables.
            campo_id (str): el nombre del campo común en X y Y para unirlos.
            df (DataFrame): la unión de los dos anteriores.
    """
    
    def __init__(self, Y, covariables, campo_id):
        self.Y = Y
        self.Y_nombre = Y.columns[-1]
        self.X = covariables
        self.X_nombres = [x for x in covariables.columns if x != campo_id]
        self.campo_id = campo_id
        self.df = self.__merge_covars()
        
    def __merge_covars(self):
        df = (self.Y.merge(self.X, on=self.campo_id)
                    .replace([np.inf, -np.inf], np.nan)
                    .dropna())
        return df
    
    def displot_Y(self, size=(12,6)):
        """Regresa el histograma de la variable dependiente."""
        f, ax = plt.subplots(1,figsize=size)
        ax = sns.histplot(data=self.Y, x=self.Y_nombre, ax=ax)
        ax.axvline(x=self.Y[self.Y_nombre].mean(), color='red')
        ax.set_ylabel("Conteo")
        return ax
    
    def pairplot_X(self, altura=10, ratio=1):
        """Regresa un pairplot de las variables independientes."""
        return sns.pairplot(self.X)
    
    def describe_Y(self):
        """Regresa un DataFrame con estadísticas descriptivas de la variable dependiente."""
        d = self.Y[self.Y_nombre].describe()
        v = pd.Series({"Var":self.Y[self.Y_nombre].var()})
        d = d.append(v)
        d = pd.DataFrame(d)
        d = d.reset_index()
        d.columns = ['Estadístico', '']
        d = d.set_index('Estadístico')
        orden = ['count', 'mean','Var',  'std', 'min', '25%', '50%', '75%', 'max']
        d = d.reindex(orden)
        d = d.rename({'count': 'N', 'mean': 'Media', 
                      'Var':'Varianza', 
                      'std': 'Desviación estándar',
                      'min': 'Mínimo',
                      'max': 'Máximo'})
        return d
        
    def mapa_Y(self, agregacion, ax=None, 
               size=(10,10), clasificacion='quantiles', 
               cmap='YlOrRd', legend=True):
        """ Regresa un ax con el mapa de la variable dependiente.
        
            Args:
            agregacion (str): colonias/cuadrantes
            ax (matplotlib.plot.ax): el eje en donde se hace el mapa (opcional, default None)
            size ((int,int)): tamaño del mapa (opcional, si se pasa un eje se ignora)
            clasificacion (str): esquema de clasificación demapclassify (opcional)
            cmap (str): mapa de colores de matplotlib (opcional)
            legend (bool): poner o no poner la leyenda (opcional)
        """
        capa = gpd.read_file("datos/criminologia_capas.gpkg", layer=agregacion)
        capa = capa.merge(self.Y, on=self.campo_id)
        if ax is None:
            f, ax = plt.subplots(1,figsize=size)
        ax = capa.plot(self.Y_nombre, scheme=clasificacion, ax=ax, cmap=cmap, legend=legend)
        ax.set_axis_off()
        ax.set_title(self.Y_nombre)
        return ax
    
    def mapa_X(self, covariable, agregacion, ax=None, 
               size=(10,10), clasificacion='quantiles', 
               cmap='YlOrRd', legend=True):
        """ Regresa un ax con el mapa de la variable dependiente.
        
            Args:
            covariable (str): Nombre de la columna en X para hacer el mapa
            agregacion (str): colonias/cuadrantes
            ax (matplotlib.plot.ax): el eje en donde se hace el mapa (opcional, default None)
            size ((int,int)): tamaño del mapa (opcional, si se pasa un eje se ignora)
            clasificacion (str): esquema de clasificación demapclassify (opcional)
            cmap (str): mapa de colores de matplotlib (opcional)
            legend (bool): poner o no poner la leyenda (opcional)
        """
        capa = gpd.read_file("datos/criminologia_capas.gpkg", layer=agregacion)
        capa = capa.merge(self.X, on=self.campo_id)
        if ax is None:
            f, ax = plt.subplots(1,figsize=size)
        ax = capa.plot(covariable, scheme=clasificacion, ax=ax, cmap=cmap, legend=legend)
        ax.set_axis_off()
        ax.set_title(covariable)
        return ax
        
    # TODO: 
    # agregar/quitar variables
    # checar que exista el campo_id en las dos bases
    # quitamos unas filas, hay que llevar registro de eso
    # Calcular variables con retraso espacial
    # Implementar transformadores sobre las variables
    

In [ ]:
usos = get_uso_de_suelo()
usos = agrega_uso_suelo(usos, unidades='colonias')
ca = CapaDeAnalisis(Y, usos, 'colonia_cve')
assert ca.X.equals(usos)
assert ca.Y.equals(Y)
assert type(ca.describe_Y()) == pd.DataFrame
# TODO: pruebas a las gráficas

## ModeloGLM

In [ ]:
# export
class ModeloGLM(object):
    """ Wrapper para modelos de Regresión GLM de statsmodels.
        
        La clase prepara y ajusta un modelo GLM usando la variable objetivo
        definida en CapaDeAnalisis y **todas** las covariables.
        
        Args:
            capa (CapaDeAnalisis): objeto con las variables del modelo.
            familia (statsmodels.api.families.Family()) la distribución a usar en el modelo GLM.
            nombre (str): si no es nulo, el nombre, si no, se asigna nombre al azar.
            
        Atributos:
            capa (CapaDeAnalisis): objeto con las variables del modelo.
            familia (statsmodels.api.families.Family()) la distribución a usar en el modelo GLM.
            formula (str): la fórmula usada para ajustar el modelo.
            df_resultado (DataFrame): los resultados como DataFrame.
            df_diagnostico (DataFrame): los diagnósticos como DataFrame.
            nombre (str): nombre que identifica al modelo.
        Métodos:
            fit(): Ajusta el modelo.
            grafica_de_ajuste(): regresa la gráfica de ajuste.
    """
    def __init__(self, capa, familia, nombre=None):
        self.capa = capa
        self.familia = familia
        self.nombre = self.__get_nombre(nombre)
        self.formula = self.__get_formula()
        self.__modelo = self.__get_modelo()
        self.modelo_ajustado = None
        self.df_resultado = None
        self.df_diagnostico = None

    def __get_nombre(self, nombre):
        if nombre is None:
            letters = string.ascii_lowercase
            return ''.join(random.choice(letters) for i in range(5))
        else:
            return nombre
    def __get_formula(self):
        ls = f"Q('{self.capa.Y_nombre}') ~ "
        rs = ""
        for nombre in self.capa.X_nombres:
            rs += f"Q('{nombre}') + "        
        formula = ls + rs
        formula = formula[:-3]
        return formula
    
    def __get_modelo(self):
        modelo = smf.glm(formula = self.formula,
                         data    = self.capa.df, 
                         family  = self.familia)
        return modelo
    
    def __resultados_a_df(self, resultados):
        """LLena self.df_resultado con los resultados del ajuste."""
        results_df = pd.DataFrame({"coef":resultados.params,
                                   "std err": resultados.bse,
                                   "z": resultados.tvalues,
                                   "P>|z|":resultados.pvalues,
                                   "conf_lower":resultados.conf_int()[0],
                                   "conf_higher":resultados.conf_int()[1]
                                    })
        self.df_resultado = results_df

    def __diagnostico_a_df(self, resultados):
        """LLena self.df_diagnostico con los resultados del ajuste."""
        indice = ["Log-Likelihood", "Deviance", "Pearson chi2"]
        valores = [resultados.llf, resultados.deviance, resultados.pearson_chi2]
        results_df = pd.DataFrame({"Diagnóstico": indice, "Valor": valores})
        self.df_diagnostico = results_df
        
    def fit(self):
        fm = self.__modelo.fit()
        self.modelo_ajustado = fm
        self.__resultados_a_df(fm)
        self.__diagnostico_a_df(fm)
        return fm
    
    def grafica_de_ajuste(self, size=(10,5), ax=None):
        """Regresa un ax con la gráfica de ajuste del modelo."""
        if ax is None:
            f, ax = plt.subplots(1,figsize=size)
        y =  self.capa.df[self.capa.Y_nombre].values
        ax = sns.regplot(x=self.modelo_ajustado.mu, y=y)
        ax.set_title('Gráfica de Ajuste del Modelo')
        ax.set_ylabel('Valores observados')
        ax.set_xlabel('Valores ajustados')
        return ax
    
    def grafica_residuales(self, tipo="deviance", size=(10,5), ax=None):
        """Regresa un ax con la gráfica de Dependencia de los Residuales."""
        observados = self.capa.df[self.capa.Y_nombre].values
        if tipo == "deviance":
            y = self.modelo_ajustado.resid_deviance
            y_label = "Residual (Deviance)"
        else:
            y = self.modelo_ajustado.resid_pearson
            y_label = "Residual (Pearson)"
        ax = sns.scatterplot(x=observados, y=y)
        ax.hlines(0, 0, observados.max(), colors='black')
        ax.set_title('Gráfica de Dependencia de los Residuales')
        ax.set_ylabel(y_label)
        ax.set_xlabel('Valores ajustados')
        return ax   
    
    def histograma_deviance(self, size=(10,5), ax=None):
        """Regresa un ax con el hitograma de deviance de los residuales."""
        resid = self.modelo_ajustado.resid_deviance.copy()
        resid_std = stats.zscore(resid)
        resid_std = pd.DataFrame(resid_std, columns=["Desviación"])
        if ax is None:
            f, ax = plt.subplots(1,figsize=size)
        ax = sns.histplot(data=resid_std, x="Desviación", ax=ax)
        ax.set_title('Histograma de desviación estandarizada')
        ax.set_ylabel('Conteo')
        return ax
        
        
        

In [ ]:
m = ModeloGLM(ca, sm.families.NegativeBinomial())
fm = m.fit()
assert type(m.df_resultado) == pd.DataFrame
assert type(m.df_diagnostico) == pd.DataFrame

## ComparaModelos

In [ ]:
# export
class ComparaModelos(object):
    """ Clase para construir comparaciones de modelos.
        Construte dos DataFrames para visualizar rápidamente una comparación de los modelos:
        uno con los resultados (coeficientes, etc) y otro con los diagnósticos.
        Args:
            modelos (list): Lista de modelos a comparar
            columnas (list): Lista de columnas que aparecen en la comparación de resultados 
                             (opcional, default None)
            redondeo: (int): Decimales a usar en la comparaciónn (opcional, default None)
        Atributos:
            modelos (list): Lista de modelos a comparar
            comparacion (DataFrame): comparación de los resultados
    """
    def __init__(self, modelos, columnas=None, redondeo=None):
        self.modelos = modelos
        self.comparacion = self.__une_resultados(columnas, redondeo)
        self.diagnosticos = self.__une_diagnosticos(redondeo)
    def __une_resultados(self, columnas, redondeo):
        if columnas is None:
            unidos = reduce(lambda left, right: 
                            left.df_resultado.join(right.df_resultado, 
                                                   how='outer', 
                                                   lsuffix="-" + left.nombre,
                                                   rsuffix="-" + right.nombre)
                            ,self.modelos)
        else:
            dfs = [(m.df_resultado, m.nombre) for m in self.modelos]
            dfs = [(df[0][columnas], df[1]) for df in dfs]
            unidos = reduce(lambda left, right: 
                            left[0].join(right[0], 
                                      how='outer', 
                                      lsuffix="-" + left[1],
                                      rsuffix="-" + right[1])
                            ,dfs)
            
        orden = [list(m.df_resultado.index) for m in self.modelos]
        orden = list(chain.from_iterable(orden))
        unidos = unidos.reindex(orden)
        if columnas is None:
            valores = self.modelos[0].df_resultado.columns
        else:
            valores = columnas
        nombres = ["Modelo " + m.nombre for m in self.modelos]
        indice_nuevo = pd.MultiIndex.from_product([nombres, valores])
        unidos.columns = indice_nuevo
        if redondeo:
            unidos = unidos.round(redondeo)
        return  unidos
    
    def __une_diagnosticos(self, redondeo):
        unidos = reduce(lambda left, right: 
                        left.df_diagnostico.merge(right.df_diagnostico, 
                                                  on="Diagnóstico",
                                                  suffixes=(" Modelo " + left.nombre, 
                                                            " Modelo " + right.nombre)),
                        self.modelos)
        if redondeo:
            unidos = unidos.round(redondeo)
        return unidos
                        
        

In [ ]:
# variables para el segundo modelo
diccionario = get_diccionario_censo()
censo = get_variables_censo()
agregado = agrega_en_unidades(censo, diccionario)
agregado = censo_a_tasas(agregado, diccionario)
vars_viv = agregado[['VIVPAR_HAB', 'VPH_3YMASC']]
vars_indice = ['P5_HLI', 'POB_AFRO', 'PCON_DISC', 'P3A5_NOA', 
               'P6A11_NOA', 'P12A14NOA', 'P15YM_AN', 'PSINDER', 'PDESOCUP']
indice = IndicePCA(agregado, vars_indice)
indice.calcula_indice()
var_m1 = (vars_viv
          .join(indice.indice.set_index('colonia_cve'))
          .rename({'Índice': 'Concentración de Desventajas'}, axis=1))
usos = get_uso_de_suelo()
usos = agrega_uso_suelo(usos, unidades='colonias')
var_m2 = var_m1.join(usos)
# Dos capas de análisis
ca1 = CapaDeAnalisis(Y, var_m1, 'colonia_cve')
ca2 = CapaDeAnalisis(Y, var_m2, 'colonia_cve')
# Ajustamos modelos
m1 = ModeloGLM(ca1, sm.families.NegativeBinomial(),"nulo")
r1 = m1.fit()
m2 = ModeloGLM(ca2, sm.families.NegativeBinomial(), "uso de suelo")
r2 = m2.fit()
# Probamos la comparacion
compara = ComparaModelos([m1,m2])
assert type(compara.comparacion) == pd.DataFrame
assert type(compara.diagnosticos) == pd.DataFrame
compara = ComparaModelos([m1, m2], ['coef', 'P>|z|'], 3)
assert type(compara.comparacion) == pd.DataFrame
assert type(compara.diagnosticos) == pd.DataFrame